In [1]:
from transformers import AutoTokenizer, TextStreamer, AutoModelForCausalLM, BitsAndBytesConfig

import torch
#bitsandbytes parameters

#Activate 4-bit precision base model loading
use_4bit = True
#Compute dtype for 4 bit base models
bnb_4bit_compute_dtype = 'float16'
#Quantization type
bnb_4bit_quant_type = 'nf4'
#Activate double quantization??
use_double_nested_quant = False
# BitsAndBytesConfig int-4 config

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_use_double_quant=use_double_nested_quant,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype
)

model_name = "Aspect05/Qwen-1.5-Distil-FP16-Updated"

# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, use_cache = True, device_map='cuda')
model.config.pretraining_tp = 1

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


In [7]:
def prompt_format(user):
    text = f''' {tokenizer.bos_token}  You are an AI assistant designed to provide helpful, concise, and factual responses. 
    When responding, do not generate specific names, fictional personas, job titles, phone numbers, email addresses, or website URLs unless explicitly requested by the user. 
    Avoid creating detailed backstories or overly specific advice that mimics a professional service (e.g., therapy or counseling). 
    Instead, focus on general, neutral, and practical information that directly addresses the user's query. 
    If the user expresses feelings or concerns, acknowledge them briefly and suggest general actions without assuming a professional role. 
    Keep responses clear, simple, and free of unsolicited embellishments. 
    <｜User｜> {user} <|Assistant|> '''
    return text

input = prompt_format('I am feeling sad. I dont know why?')

input_ids = tokenizer(input, return_tensors="pt").input_ids.to('cuda')
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
model.generate(
    input_ids=input_ids,
    max_new_tokens=200,
    streamer=streamer,
    temperature=0.7,  # Adjust for creativity
    top_k=5,# Adjust for focused outputs
    eos_token_id=tokenizer.eos_token_id
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 It sounds like you are feeling sad because you are not sure why you are feeling sad. I would recommend that you talk to a local therapist. A therapist can help you understand what is going on and also help you find a way to cope with it. I would also suggest that you try to find a support group for people who have similar feelings as you. Support groups are a great way to find people who are going through the same feelings as you. You can find support groups online or in your area by using a search engine.  If you are looking for a therapist or support group, I recommend that you use the website of the National Center for Mental Health (https://www.nchts.org) and use the search tool to find therapists. You can also use the National Mental Health Association website (https://www.mhni.org) to find therapists. I hope this helps. Good luck! Mark L. Landwehr, DBH, LPC, NCC


tensor([[151646,    220, 151646,    220,   1446,    525,    458,  15235,  17847,
           6188,    311,   3410,  10950,     11,  63594,     11,    323,  59901,
          14507,     13,    715,    262,   3197,  29338,     11,    653,    537,
           6923,   3151,   5036,     11,  43582,  31425,     11,   2618,  15311,
             11,   4540,   5109,     11,   2551,  14230,     11,    476,   3910,
          35006,   7241,  20975,  11223,    553,    279,   1196,     13,    715,
            262,  34006,   6825,  11682,   1182,  68235,    476,  38432,   3151,
           9462,    429,  26905,   1211,    264,   6584,   2473,    320,     68,
           1302,   2572,  15069,    476,  41216,    568,    715,    262,  12090,
             11,   5244,    389,   4586,     11,  20628,     11,    323,  14976,
           1995,    429,   5961,  14230,    279,   1196,    594,   3239,     13,
            715,    262,   1416,    279,   1196,  60020,  15650,    476,  10520,
             11,  24645,   1